In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import argparse


Langchain Import

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import GooglePalmEmbeddings
from langchain.vectorstores import Pinecone
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_core.prompts import PromptTemplate


load Question/Answer dataset

In [3]:
df1 = pd.read_csv('python_mcq_bank_100.csv')
df2 = pd.read_csv('math_mcq_bank_100.csv')
df3 = pd.read_csv('science_mcq_bank_100.csv')

In [4]:
df1

question  \
0   What is the correct file extension for Python ...   
1        Which symbol is used for comments in Python?   
2                   What is the output of print(2+3)?   
3   Which function is used to display output in Py...   
4   Which keyword is used to define a variable in ...   
..                                                ...   
95   What does the tell() method do in file handling?   
96  Which of the following is true for Python's GI...   
97  Which of the following is an example of an imm...   
98            What is the output of print(2 + 3 * 5)?   
99  What is the correct way to define an empty lis...   

                                          option_a  \
0                                              .pt   
1                                               //   
2                                               23   
3                                          print()   
4                                              let   
..                                             ...   
95               Returns the current file position   
96  Allows true parallel CPU bound multi-threading   
97                                            list   
98                                              25   
99                                          list()   

                                             option_b  \
0                                                 .py   
1                                                   #   
2                                                   5   
3                                              echo()   
4                                                 var   
..                                                ...   
95                        Returns the end of the file   
96  Prevents multiple native threads from executin...   
97                                         dictionary   
98                                                 17   
99                                                 {}   

                        option_c                option_d correct_answer  \
0                           .pyt                 .python              B   
1                             /*                      --              B   
2                              6                   Error              B   
3                      display()                 write()              A   
4                         define       No keyword needed              D   
..                           ...                     ...            ...   
95  Returns the size of the file   Returns the file name              A   
96  Only applies to Python 3.11+  Is a garbage collector              B   
97                           set               frozenset              D   
98                             7                      10              B   
99                            []                     NaN              C   

   difficulty  
0   Very Easy  
1   Very Easy  
2   Very Easy  
3   Very Easy  
4   Very Easy  
..        ...  
95     Medium  
96       Hard  
97     Medium  
98       Easy  
99       Easy  

[100 rows x 7 columns]

In [5]:
df2

question            option_a  \
0   What is the value of pi (π) rounded to two dec...                3.14   
1   What is the next number in the sequence: 2, 4,...                  20   
2             If 4x - 2 = 10, what is the value of x?                   2   
3           Which of the following is a prime number?                   9   
4   In a right-angled triangle, the square of the ...   Euclidean Theorem   
..                                                ...                 ...   
95  The equation for an ellipse is related to the ...            Vertices   
96  If a triangle has angle measures $30^{\circ}$ ...          30^{\circ}   
97              What is the additive inverse of $-5$?                  -5   
98  The study of shapes, sizes, positions, and pro...             Algebra   
99  What is the formula for the $n^{th}$ term of a...  a_n = a_1 + (n-1)d   

               option_b         option_c               option_d  \
0                  3.16             3.10                   3.15   
1                    24               32                     48   
2                     3                4                      6   
3                    15               17                     21   
4   Pythagorean Theorem  Thales' Theorem  Fermat's Last Theorem   
..                  ...              ...                    ...   
95                 Foci      Directrices                Centers   
96           60^{\circ}       90^{\circ}            180^{\circ}   
97                    0                5                    1/5   
98         Trigonometry         Geometry               Calculus   
99    a_n = a_1 r^{n-1}   a_n = n(n+1)/2              a_n = 2^n   

   correct_answer difficulty  
0               A       Easy  
1               C       Easy  
2               B     Medium  
3               C       Easy  
4               B     Medium  
..            ...        ...  
95              B       Hard  
96              C       Easy  
97              C       Easy  
98              C  Very Easy  
99              A       Hard  

[100 rows x 7 columns]

In [6]:
df3

question             option_a  \
0              What is the chemical symbol for water?                   Wo   
1          Which planet is known as the "Red Planet"?              Jupiter   
2   What is the process by which plants make their...          Respiration   
3      What unit is used to measure electric current?                 Watt   
4   Which organ pumps blood throughout the human b...                Lungs   
..                                                ...                  ...   
95  Which layer of the atmosphere contains the ozo...          Troposphere   
96              What is the chemical symbol for Gold?                   Go   
97                             What causes a rainbow?  Reflection of light   
98  The process of a cell engulfing a solid partic...          Pinocytosis   
99              Which instrument measures wind speed?            Barometer   

                                 option_b             option_c  \
0                                      Wa                  H₂O   
1                                    Mars                Venus   
2                           Transpiration       Photosynthesis   
3                                    Volt                  Ohm   
4                                 Kidneys                Brain   
..                                    ...                  ...   
95                           Stratosphere           Mesosphere   
96                                     Au                   Ag   
97  Refraction and dispersion of sunlight  Scattering of light   
98                             Exocytosis         Phagocytosis   
99                             Anemometer           Hygrometer   

                 option_d correct_answer difficulty  
0                     O2H              C       Easy  
1                  Saturn              B       Easy  
2           Decomposition              C       Easy  
3                  Ampere              D     Medium  
4                   Heart              D  Very Easy  
..                    ...            ...        ...  
95           Thermosphere              B       Hard  
96                     Gd              B       Easy  
97  Interference of light              B     Medium  
98                Osmosis              C       Hard  
99            Seismograph              B     Medium  

[100 rows x 7 columns]

In [7]:
import json

import json


In [8]:
quiz = 'Quiz_Progress.json'

In [9]:

try:
    
   
    with open(quiz, 'r') as f:
        quiz_data = json.load(f)

   
    incorrect_answers = quiz_data.get("incorrectlyAnswered")

    # 3. Format the data into a single, clean string for the agent
    if not incorrect_answers:
        print("No incorrect answers found in the report.")
        rag_input_string = "The student answered all questions correctly."
    else:
        rag_input_string = "\n".join([
            (f"- Topic: {ans.get('topic', 'N/A')}, "
             f"Difficulty: {ans.get('difficulty', 'N/A')}, "
             f"Question: \"{ans['questionText']}\", "
             f"Their Incorrect Answer: '{ans['userAnswer']}', "
             f"Correct Answer: '{ans['correctAnswer']}'")
            for ans in incorrect_answers
        ])
    
    # 4. Print the final string that is ready for the RAG agent
    print("\n--- Input Ready for RAG Agent ---")
    print(rag_input_string)



except FileNotFoundError:
    print(f"Error: The file '{quiz}' was not found.")
except json.JSONDecodeError:
    print(f"Error: The file '{quiz}' is empty or does not contain valid JSON. Please check the file content.")
except KeyError:
    print("Error: The JSON file is missing the 'incorrectlyAnswered' key.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")




--- Input Ready for RAG Agent ---
- Topic: Algebra, Difficulty: Medium, Question: "If 4x - 2 = 10, what is the value of x?", Their Incorrect Answer: 'A', Correct Answer: 'B'
- Topic: Geometry, Difficulty: Very Easy, Question: "The formula for the area of a rectangle is:", Their Incorrect Answer: 'B', Correct Answer: 'A'
- Topic: Geometry, Difficulty: Easy, Question: "A triangle with all sides of equal length is called:", Their Incorrect Answer: 'B', Correct Answer: 'C'
- Topic: Trigonometry, Difficulty: Hard, Question: "The value of sin(90 degrees) is:", Their Incorrect Answer: 'C', Correct Answer: 'A'
- Topic: Logarithms, Difficulty: Medium, Question: "The value of log_10(100) is:", Their Incorrect Answer: 'D', Correct Answer: 'B'
- Topic: Sequences, Difficulty: Hard, Question: "What is the formula for the nth term of an arithmetic sequence?", Their Incorrect Answer: 'C', Correct Answer: 'A'
- Topic: Geometry, Difficulty: Hard, Question: "The equation for an ellipse is related to the

In [10]:
import load_dotenv

In [11]:
def prepare_rag_input(quiz_progress_file):
    """
    Loads the quiz progress JSON and extracts the data needed for the RAG agent.
    """
    print(f"--- [Step 1] Loading quiz data from '{quiz_progress_file}' ---")
    try:
        with open(quiz_progress_file, 'r') as f:
            quiz_data = json.load(f)

        incorrect_answers = quiz_data.get("incorrectlyAnswered")

        if not incorrect_answers:
            print("No incorrect answers found. Student passed perfectly!")
            return None # Return None to signal no work is needed

        # This formatted string is the final output of this step.
        rag_input_string = "\n".join([
            (f"- Topic: {ans.get('topic', 'N/A')}, "
             f"Difficulty: {ans.get('difficulty', 'N/A')}, "
             f"Question: \"{ans['questionText']}\", "
             f"Their Incorrect Answer: '{ans['userAnswer']}', "
             f"Correct Answer: '{ans['correctAnswer']}'")
            for ans in incorrect_answers
        ])
        print("--- [Step 1] Successfully prepared input for the RAG agent. ---")
        return rag_input_string

    except FileNotFoundError:
        print(f"Error: The file '{quiz_progress_file}' was not found.")
        return "Error"
    except json.JSONDecodeError:
        print(f"Error: The file '{quiz_progress_file}' is empty or invalid.")
        return "Error"
    except Exception as e:
        print(f"An unexpected error occurred during file processing: {e}")
        return "Error"

In [26]:
import os
import json
from dotenv import load_dotenv

# --- AI Integration Imports ---
# IMPORTANT: Make sure you have run "pip install --upgrade langchain langchain-openai"
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent, Tool
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_core.prompts import PromptTemplate

# ==============================================================================
# STEP 1: LOAD & PREPARE THE DATA
# ==============================================================================
def prepare_rag_input(quiz_progress_file):
    """
    Loads the quiz progress JSON and formats the incorrect answers for the agent.
    """
    print(f"--- [Step 1] Loading quiz data from '{quiz_progress_file}' ---")
    try:
        with open(quiz_progress_file, 'r') as f:
            quiz_data = json.load(f)
        incorrect_answers = quiz_data.get("incorrectlyAnswered")
        if not incorrect_answers:
            print("No incorrect answers found. Student passed perfectly!")
            return None
        rag_input_string = "\n".join([
            (f"- Topic: {ans.get('topic', 'N/A')}, "
             f"Difficulty: {ans.get('difficulty', 'N/A')}, "
             f"Question: \"{ans['questionText']}\", "
             f"Their Incorrect Answer: '{ans['userAnswer']}', "
             f"Correct Answer: '{ans['correctAnswer']}'")
            for ans in incorrect_answers
        ])
        print("--- [Step 1] Successfully prepared input for the RAG agent. ---")
        return rag_input_string
    except FileNotFoundError:
        print(f"Error: The file '{quiz_progress_file}' was not found.")
        return "Error"
    except json.JSONDecodeError:
        print(f"Error: The file '{quiz_progress_file}' is empty or invalid.")
        return "Error"
    except Exception as e:
        print(f"An unexpected error occurred during file processing: {e}")
        return "Error"

# ==============================================================================
# STEP 2: CREATE THE RAG AGENT
# ==============================================================================
def create_rag_agent():
    """
    Builds and returns a LangChain agent powered by OpenAI and Google Search.
    """
    print("--- [Step 2] Initializing the RAG Agent with OpenAI ---")
    
    # 2a. Define the tool(s) the agent can use. This is the "Retrieval" part.
    # The agent can only use the tools you provide here.
    search = GoogleSearchAPIWrapper()
    tools = [
        Tool(
            name="Google Search",
            func=search.run,
            description="Use this to find specific learning resources like YouTube videos, articles, PDFs, and online courses for a technical concept.",
        )
    ]
    
    # 2b. Define the agent's instructions (the prompt).
    # This is a critical step that tells the agent how to reason.
    agent_prompt_template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Now, begin!

Question: {input}
Thought: {agent_scratchpad}"""
    
    prompt = PromptTemplate.from_template(agent_prompt_template)
    
    # 2c. Define the "brain" of the agent, which is the Large Language Model.
    # We are using OpenAI's powerful gpt-4o model.
    llm = ChatOpenAI(model="gpt-4o", temperature=0.7)
    
    # 2d. Create the agent by combining the LLM, tools, and prompt.
    agent = create_react_agent(llm, tools, prompt)
    
    # The AgentExecutor is the runtime that makes the agent work.
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
    
    print("--- [Step 2] RAG Agent created successfully. ---")
    return agent_executor

# ==============================================================================
# STEP 3: RUN THE FULL PIPELINE
# ==============================================================================
if __name__ == "__main__":
    
    # Try to load API keys from a .env file first.
    load_dotenv()
    
    # Check for the OpenAI key. If not found, prompt the user.
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("OpenAI API Key not found in .env file.")
        api_key = input("Please enter your OPENAI_API_KEY: ")
        os.environ["OPENAI_API_KEY"] = api_key

    # Check for the Google Search CSE ID. If not found, prompt the user.
    cse_id = os.getenv("GOOGLE_CSE_ID")
    if not cse_id:
        print("Google Search Engine ID not found in .env file.")
        cse_id = input("Please enter your GOOGLE_CSE_ID: ")
        os.environ["GOOGLE_CSE_ID"] = cse_id

    # --- Run Step 1 ---
    quiz_file = 'quiz_progress.json'
    rag_input = prepare_rag_input(quiz_file)
    
    if rag_input and rag_input != "Error":
        # --- Run Step 2 ---
        rag_agent = create_rag_agent()
        
        # --- Execute the Agent ---
        print("\n--- [Step 3] Invoking Agent to Generate Recommendations... ---")
        try:
            # 3a. Define the detailed mission for the agent.
            # We combine our instructions with the student's specific incorrect answers.
            # This becomes the "Question" that the agent will work on.
            final_question = f"""
            You are an expert, friendly, and encouraging academic tutor. Your mission is to create a comprehensive and diverse learning plan for a student based on the exact questions they answered incorrectly.

            For EACH incorrect question provided below, perform these steps:
            1.  Identify the core academic concept being tested.
            2.  Use Google Search to find high-quality learning resources for that specific concept.
            3.  Recommend ONE of each of the following if you find a good match:
                * A Video Tutorial (from YouTube, etc.)
                * A Written Guide (an article or PDF)
                * An Online Course (from Coursera, freeCodeCamp, etc.)
            4.  For each resource, provide the title, a one-sentence description, and the URL.

            Combine all recommendations into a single, well-formatted Markdown response.

            Here are the incorrect questions:
            {rag_input}
            """
            
            # 3b. Run the agent. The agent will now start its "Thought -> Action -> Observation" loop.
            response = rag_agent.invoke({"input": final_question})
            
            # 3c. Extract the final answer from the agent's response.
            final_recommendations = response["output"]
        except Exception as e:
            final_recommendations = f"An error occurred while running the agent: {e}"

        # --- Final Output ---
        print("\n\n--- ✅ AI Tutor's Final Recommendation ---")
        print(final_recommendations)
    elif rag_input == "Error":
        print("\nPipeline stopped due to an error during data loading.")
    else:
         print("\n\n--- ✅ AI Tutor's Final Recommendation ---")
         print("Great job! The student answered all questions correctly.")


### Next Steps for You:





--- [Step 1] Loading quiz data from 'quiz_progress.json' ---
--- [Step 1] Successfully prepared input for the RAG agent. ---
--- [Step 2] Initializing the RAG Agent with OpenAI ---
--- [Step 2] RAG Agent created successfully. ---

--- [Step 3] Invoking Agent to Generate Recommendations... ---


> Entering new AgentExecutor chain...


--- ✅ AI Tutor's Final Recommendation ---
An error occurred while running the agent: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
